In [35]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

ct_data = pd.read_csv(r"C:\Users\Yeray\Desktop\DATA_SCIENCE_ML\IG-Engagemente-Insights\IG-Engagement-Insights\ANALYSIS_IG_ACCOUNT\DATA_CT\CT_SOLO_ACTUAL.csv")
print(ct_data.info())

# Objetivos a conseguir con esta exploración inicial
# 1- Conocer la estructura y tipo de de datos con los que vamos a trabajar, además de un análisis descriptivo de los mismos.
# 2- Avergiuar que columnas no nos proporcionan información relevante y cuales si y reordenar el conjunto de datos.
# 3- Sacar información como distribución de valores nulos, outliers y todo lo típico de cara a la limpieza de datos.
# 4- Visualizar datos para sacar conclusiones preliminares.
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 500)
pd.set_option('display.show_dimensions', True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Columns: 400 entries, alt to videoViewCount
dtypes: bool(2), float64(162), int64(10), object(226)
memory usage: 127.7+ KB
None


In [36]:
from tabulate import tabulate
print(ct_data.shape)
head_df = ct_data.head(5)

print(tabulate(head_df, headers=head_df.columns, tablefmt='fancy_grid'))

(41, 400)
╒════╤═══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

In [37]:
#También vamos a eliminar la publicación SORTEO, pues es un tipo de publicación que no nos interesa analizar ya que independientemente de la hora a la que se publique va a tener una alta tasa de interacción.
ct_data_copy = ct_data.copy()

post_sorteo = ct_data_copy['caption'].str.contains('SORTEO')
#print(post_sorteo)
ct_data_copy = ct_data_copy[~post_sorteo]
print(ct_data_copy.shape)

#Vamos a ver los valores nulos que tenemos en el dataset y el porcentaje de valores nulos que hay en cada columna.
nulls = ct_data_copy.isnull().sum()
nulls = nulls[nulls > 0]
nulls = nulls/ct_data_copy.shape[0] 
print(nulls.sort_values(ascending=False))

(40, 400)
latestComments/8/owner/id                           1.000
latestComments/8/owner/profile_pic_url              1.000
childPosts/7/timestamp                              1.000
childPosts/4/firstComment                           1.000
latestComments/7/owner/username                     1.000
latestComments/7/ownerProfilePicUrl                 1.000
latestComments/7/ownerUsername                      1.000
childPosts/8/caption                                1.000
childPosts/4/caption                                1.000
latestComments/8/owner/is_verified                  1.000
childPosts/3/timestamp                              1.000
childPosts/4/ownerId                                1.000
latestComments/8/owner/username                     1.000
childPosts/3/ownerId                                1.000
childPosts/3/likesCount                             1.000
latestComments/8/ownerProfilePicUrl                 1.000
childPosts/3/firstComment                           1.000
late

In [38]:
#Vamos a eliminar aquellas columnas que solo tienen valores nulos/vacíos/0 
all_nan_cols = [i for i in ct_data_copy if ct_data_copy[i].isnull().all()]
all_0_cols = [i for i in ct_data_copy if (ct_data_copy[i] == 0).all()]
all_vacio_cols = [i for i in ct_data_copy if (ct_data_copy[i] == '').all()]
print(all_nan_cols)
print(all_0_cols)
print(all_vacio_cols)
ct_data_copy = ct_data_copy.drop(all_nan_cols, axis=1)
ct_data_copy = ct_data_copy.drop(all_0_cols, axis=1)
ct_data_copy = ct_data_copy.drop(all_vacio_cols, axis=1)
print(ct_data_copy.shape)

['childPosts/0/caption', 'childPosts/0/firstComment', 'childPosts/0/likesCount', 'childPosts/0/ownerId', 'childPosts/0/timestamp', 'childPosts/1/caption', 'childPosts/1/firstComment', 'childPosts/1/likesCount', 'childPosts/1/ownerId', 'childPosts/1/timestamp', 'childPosts/2/caption', 'childPosts/2/firstComment', 'childPosts/2/likesCount', 'childPosts/2/ownerId', 'childPosts/2/timestamp', 'childPosts/3/caption', 'childPosts/3/firstComment', 'childPosts/3/likesCount', 'childPosts/3/ownerId', 'childPosts/3/timestamp', 'childPosts/4/caption', 'childPosts/4/firstComment', 'childPosts/4/likesCount', 'childPosts/4/ownerId', 'childPosts/4/timestamp', 'childPosts/5/caption', 'childPosts/5/firstComment', 'childPosts/5/likesCount', 'childPosts/5/ownerId', 'childPosts/5/timestamp', 'childPosts/5/videoPlayCount', 'childPosts/6/caption', 'childPosts/6/firstComment', 'childPosts/6/likesCount', 'childPosts/6/ownerId', 'childPosts/6/timestamp', 'childPosts/7/caption', 'childPosts/7/firstComment', 'chil

In [39]:
#Todas aquellas variables relacionadas con las dimensiones, fotos, url, shortcode, etc no nos interesan para el análisis de engagement, por lo que las eliminamos.
#Vamos a meter todas las palabras clave en una lista y eliminar las columnas que contengan esas palabras.
palabras_clave_para_eliminar = ['dimension', 'url', 'Url', 'tagged', 'mentions', 'shortCode', 'alt', 'id', 'Id', 'username', 'Username', 'text', 'firstComment', 'is_verified',' isPinned','latestComments' ]
columnas_a_eliminar = [col for col in ct_data_copy.columns if any(palabra in col for palabra in palabras_clave_para_eliminar)]
ct_data_copy = ct_data_copy.drop(columns=columnas_a_eliminar)
print(ct_data_copy.shape)
#for col in ct_data_copy.columns:
#    if 'dimension' in col or 'url' in col or 'Url' in col or 'shortCode' in col or 'alt' in col or 'id' in col or 'Id' in col or 'username' in col or 'Username' in col:
#        ct_data_copy = ct_data_copy.drop(col, axis=1)

#También vamos a eliminar las columnas que nos indican el tipo de cada post, ya que solo nos interesa saber el tipo que es la publicación, es decir, si es imagen, video o carrusel.
for i in range(0, 9):
        ct_data_copy = ct_data_copy.drop(f'childPosts/{i}/type', axis=1)
        ct_data_copy = ct_data_copy.drop(f'childPosts/{i}/commentsCount', axis=1)
ct_data_copy = ct_data_copy.drop('productType', axis=1)
print(ct_data_copy.shape)


(40, 52)
(40, 33)


In [40]:
#Esta celda ha sido usada para ver las columnas restantes conforme vamos eliminando las que no nos interesan. 
#Vamos a ver el porcentaje de cada columna de valores nulos tras haber elimminado algunas de las columnas que no nos interesan

nulls = ct_data_copy.isnull().sum()
nulls = nulls[nulls > 0]
nulls = nulls/ct_data_copy.shape[0] 
print(nulls.sort_values(ascending=False))

hashtags/12                      0.975
images/8                         0.975
images/7                         0.950
isPinned                         0.925
hashtags/11                      0.925
images/6                         0.900
images/5                         0.875
hashtags/10                      0.875
hashtags/9                       0.775
images/4                         0.725
musicInfo/artist_name            0.700
musicInfo/should_mute_audio      0.700
musicInfo/song_name              0.700
images/3                         0.700
musicInfo/uses_original_audio    0.700
hashtags/8                       0.675
images/2                         0.675
hashtags/7                       0.600
images/0                         0.550
images/1                         0.550
hashtags/6                       0.550
hashtags/5                       0.425
hashtags/4                       0.375
hashtags/1                       0.300
hashtags/3                       0.300
hashtags/2               

In [41]:
#Tras haber revisado exhaustivamente las columnas y habernos deshecho de toda la información que no nos aportaba nada debido a todos los datos innecesarios que nos daba el scrapper,
#vamos a crear algunas columnas nuevas para algo más de información visual

#Comprobamos si la publicación tiene música o no, 1 = tiene música, 0 = no tiene música
ct_data_copy['hasMusic'] = ct_data_copy['musicInfo/song_name'].apply(lambda x: 0 if pd.isna(x) or x=='' else 1)
#head_df_2 = ct_data_copy.head(10)
#print(tabulate(head_df_2, headers=head_df_2.columns, tablefmt='fancy_grid'))

#Eliminamos toda otra columna en la que se hable de música, ya que no nos interesa para el análisis de engagement.
columns_to_drop = [col for col in ct_data_copy.columns if col.startswith('musicInfo/')]
ct_data_copy = ct_data_copy.drop(columns=columns_to_drop)
print(ct_data_copy.shape)

(40, 30)


In [42]:
from tabulate import tabulate
#Vamos a ver el número de hastags que tiene cada publicación, como tenemos una columna del 1 al n número de variables indicando el texto de cada hastag, vamos a comprobar si cada hastag tiene texto 
# y si es así contamos 1 y lo vamos sumando
def count_hashtags(row):
    total = 0
    for i in range(10):
        col = f'hashtags/{i}'
        if col in row:
            val = row[col]
            # condición: si no es nulo y no es una string vacía
            if pd.notna(val) and str(val).strip() != '':
                total += 1
    return total

ct_data_copy['num_hashtags'] = ct_data_copy.apply(count_hashtags, axis=1)

#print(ct_data_copy.head(10))


#Hacemos lo mismo para el número de imágenes que tiene cada publicación, como tenemos 3 tipos de publicación: imagen, video y sidecar, vamos a 
# declarar que 0 = video, 1 = imagen y i = sidecar siendo i= al número de imágenes que tiene la publicación.
def count_images(row):
    if row['type'] == 'Video':
        return 0
    
    if row['type'] == 'Image':
        return 1
    if row['type'] == 'Sidecar':
        total = 0
        for i in range(10):
            col = f'images/{i}'
            if col in row:
                val = row[col]
                # condición: si no es nulo y no es una string vacía
                if pd.notna(val) and str(val).strip() != '':
                    total += 1
        return total
    return 0

ct_data_copy['num_images'] = ct_data_copy.apply(count_images, axis=1)
for i in range(0, 9):
        ct_data_copy = ct_data_copy.drop(f'images/{i}', axis=1)
print(tabulate(ct_data_copy.head(10), headers=ct_data_copy.columns, tablefmt='fancy_grid'))

╒════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════╤══════════════╤══════════════╤══════════════════════╤═════════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤═══════════════╤═══════════════╤═══════════════╤════════════╤══════════════╤═══════════════════════════════════╤══════════════════════════╤═════════╤════════════╤════════════════╤══════════════╕
│    │ caption                                                                                                                             

In [43]:
#Ahora vamos a obtener todos los hashtags y guardarlos en una lissta, luego vamos a ver las veces que han sido usados y la media de likes que tienen
# de cara a ver si el uso de alguno de estos supone una gran mejora en el engagement.
columns_hashtags = [col for col in ct_data_copy.columns if col.startswith('hashtags/')]
def collect_hashtags(row):
    tags = []
    for col in columns_hashtags:
        val = row[col]
        if pd.notna(val) and str(val).strip() != '':
            tags.append(str(val).strip().lower())  # normalizamos a minúsculas
    return tags

ct_data_copy['hashtags_list'] = ct_data_copy.apply(collect_hashtags, axis=1)
print(ct_data_copy['hashtags_list'])

0                                                    []
1     [libros, acotar, rhysand, feyre, marcapaginas,...
2                                                    []
3     [vainilla, primavera, velasaromaticas, vela, d...
4                                                    []
5                                                    []
6     [stickers, cozy, libros, diseño, pegatinas, de...
7     [libros, lecturas, floral, primavera, paginas,...
8     [libros, velas, onceuponabrokenheart, lectura,...
9     [findesemana, café, libros, velasaromaticas, a...
10    [velasaromaticas, velas, melts, waxmelts, arom...
11    [fundasliterarias, libros, aesthetic, hechoama...
12                                                   []
13    [libros, velas, lectura, cozy, artesanía, hech...
14    [velas, libros, espiritualidad, cozy, fantasía...
15    [acotar, marcapaginas, libros, feyre, rhysand,...
16                                                   []
17    [libros, bookstagram, hechoamano, marcapag

In [44]:
ct_exploded = ct_data_copy.explode('hashtags_list').reset_index(drop=True)

# RENOMBRAR la columna `hashtags_list` por `hashtag`
ct_exploded = ct_exploded.rename(columns={'hashtags_list': 'hashtag'})

stats_hashtags = (
    ct_exploded
    .groupby('hashtag')['likesCount']
    .agg(['count', 'mean', 'max'])
    .sort_values('count', ascending=False)
)

print(stats_hashtags.head(20))
stats_hashtags.to_csv(r"C:\Users\Yeray\Desktop\DATA_SCIENCE_ML\IG-Engagemente-Insights\IG-Engagement-Insights\ANALYSIS_IG_ACCOUNT\DATA_CT\CT_SOLO_HASHTAGS_INFO.csv")

              count       mean  max
hashtag                            
libros           26  41.076923   84
lectura          16  40.437500   84
artesanía        14  42.071429   84
hechoamano       13  43.153846   84
cozy             11  34.545455   58
bookstagram      10  36.600000   58
marcapaginas      9  45.222222   84
regalos           8  29.500000   45
velas             6  36.666667   51
handmade          5  50.400000   76
acotar            5  61.200000   84
natural           4  40.500000   51
madera            4  47.000000   84
rhysand           3  58.333333   84
lecturacozy       3  38.333333   40
booktok           3  41.333333   58
aesthetic         3  59.333333   76
binding13         3  47.666667   76
lecturas          3  47.666667   58
primavera         3  44.666667   58

[20 rows x 3 columns]


In [45]:
#Eliminamos otras columnas más
for i in range(0, 13):
        ct_data_copy = ct_data_copy.drop(f'hashtags/{i}', axis=1)
ct_data_copy = ct_data_copy.drop('isPinned', axis=1)
ct_data_copy = ct_data_copy.drop('ownerFullName', axis=1)
print(tabulate(ct_data_copy.head(10), headers=ct_data_copy.columns, tablefmt='fancy_grid'))

╒════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════╤══════════════╤══════════════════════════╤═════════╤════════════╤════════════════╤══════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│    │ caption                                                                                                                                                                                                                                                

In [ ]:
#Vamos a separar la fecha en que día de la semana fue y la hora de la publicación
ct_data_copy['timestamp'] = pd.to_datetime(ct_data_copy['timestamp'])

dias_semana = {
    0: 'Lunes',
    1: 'Martes',
    2: 'Miércoles',
    3: 'Jueves',
    4: 'Viernes',
    5: 'Sábado',
    6: 'Domingo'
}
#Mediante .dayofweek obtenemos el día de la semana en formato numérico, por lo que vamos a mapear esos números a los días de la semana
ct_data_copy['dia_semana'] = ct_data_copy['timestamp'].dt.dayofweek.map(dias_semana)

meses = {
    1: 'Enero',
    2: 'Febrero',
    3: 'Marzo',
    4: 'Abril',
    5: 'Mayo',
    6: 'Junio',
    7: 'Julio',
    8: 'Agosto',
    9: 'Septiembre',
    10: 'Octubre',
    11: 'Noviembre',
    12: 'Diciembre'
}

ct_data_copy['mes'] = ct_data_copy['timestamp'].dt.month.map(meses)
ct_data_copy['year'] = ct_data_copy['timestamp'].dt.year
ct_data_copy['fecha'] = ct_data_copy['timestamp'].dt.strftime('%Y-%m-%d')
#Separamos la hora de la publicación
ct_data_copy['hora'] = ct_data_copy['timestamp'].dt.strftime('%H:%M:%S')
ct_data_copy = ct_data_copy.drop('timestamp', axis=1)



In [47]:
#Creamos la columna id para poder tener un índice de cada publicación.
ct_data_copy['id'] = ct_data_copy.index
print(tabulate(ct_data_copy.head(10), headers=ct_data_copy.columns, tablefmt='fancy_grid'))

╒════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════╤══════════════╤═════════╤════════════╤════════════════╤══════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤══════════════╤═══════════╤════════╤════════════╤══════════╤══════╕
│    │ caption                                                                                                                                                                                                        

In [30]:
#Ya hemos realizado un formateo del csv y nos hemos quedado con lo más importante, aunque posiblemente añadamos más variables relacionadas con estadísticas en un futuro. 
# Por último toca clasificar que tipo de publicación es cada post, pues nos interesa saber que producto o tipo de publicación iene más engagement y así aumentar el foco en este para aumentar las ventas.

#La idea inicial entrenar un modelo de clasificación para predecir el tipo de publicación, pero no tenemos la suficiente cantidad de datos para hacerlo. 
#Dado que tenemos pocos posts podriamos hacerlo a mano, pero esto no es una buena práctica pues la idea es que este proyecto pueda ser aplicado a cualquier cuenta sin importar su cantidad de publicaciones.
#Para ellos vamos a hacer uso de una api open source de un LLM que mediante un prompt se encarge de leer la descripción de la publicación y clasificarla.

#Vamos a sacr el csv del id de la publicación y su descripción para poder clasificarlo con el modelo de clasificación.
ct_data_copy_prompt = ct_data_copy[['id', 'caption']]
ct_data_copy_prompt.to_csv(r"C:\Users\Yeray\Desktop\DATA_SCIENCE_ML\IG-Engagemente-Insights\IG-Engagement-Insights\ANALYSIS_IG_ACCOUNT\DATA_CT\CT_SOLO_PROMPT.csv", index=False)

In [85]:
output = pd.read_csv(r"C:\Users\Yeray\Desktop\DATA_SCIENCE_ML\IG-Engagemente-Insights\IG-Engagement-Insights\output\clasificacion_publicaciones.csv", sep=',', on_bad_lines='warn')
output = output.drop('caption', axis=1)
print(output.head(10))
output.to_csv(r"C:\Users\Yeray\Desktop\DATA_SCIENCE_ML\IG-Engagemente-Insights\IG-Engagement-Insights\ANALYSIS_IG_ACCOUNT\DATA_CT\CT_SOLO_CLASIFICADO.csv", index=False, encoding='utf-8-sig')

  id tipo_de_publicacion tipo_de_producto
0  0           novedades            velas
1  1           promocion     marcapaginas
2  2           novedades    sujetapaginas
3  3           novedades            velas
4  4           promocion        multitipo
5  5           promocion            velas
6  6           novedades         stickers
7  7           novedades    sujetapaginas
8  8           promocion            velas
9  9           promocion            velas

[10 rows x 3 columns]


C:\Users\Yeray\AppData\Local\Temp\ipykernel_44216\3670131913.py:1: ParserWarning: Skipping line 15: expected 4 fields, saw 5
Skipping line 30: expected 4 fields, saw 6

  output = pd.read_csv(r"C:\Users\Yeray\Desktop\DATA_SCIENCE_ML\IG-Engagemente-Insights\IG-Engagement-Insights\output\clasificacion_publicaciones.csv", sep=',', on_bad_lines='warn')


In [31]:
#Vamos a dividir la hora de cada día en franjas horarias para un mejor orden y visualización de los datos.
print(ct_data_copy['hora'].unique())

ct_data_copy['hora'] = pd.to_datetime(ct_data_copy['hora'], format='%H:%M:%S').dt.time
ct_data_copy['minutos_desde_medianoche'] = ct_data_copy['hora'].apply(lambda x: x.hour * 60 + x.minute)

bins = [
    0,    # 00:00
    525,  # 08:45
    555,  # 09:15
    585,  # 09:45
    630,  # 10:30
    690,  # 11:30
    750,  # 12:30
    810,  # 13:30
    1020, # 17:00
    1080, # 18:00
    1140,
    1200,  # 19:00
    1440  # 24:00
]
#El rango se establece de manera de que se "redondea" a la hora más cercana o se mete dentro de un rango para aquellas horas que tienen menos publicaciones

labels = [
    'antes de 8:45',
    '9:00',
    '9:30',
    '10:00',
    '11:00',
    '12:00',
    '13:00',
    '14:00',
    '17-18',
    '18-19',
    '19-20',
    '20+'
]
ct_data_copy['franja_horaria'] = pd.cut(ct_data_copy['minutos_desde_medianoche'], bins=bins, labels=labels, right=False)
ct_data_copy = ct_data_copy.drop('minutos_desde_medianoche', axis=1)
print(ct_data_copy[['hora', 'franja_horaria']])

['09:26:29' '19:47:30' '12:51:46' '16:13:56' '09:59:46' '17:00:11'
 '11:12:48' '09:26:20' '17:01:16' '09:24:03' '08:45:49' '09:16:59'
 '09:13:08' '10:36:33' '08:36:29' '10:59:52' '09:51:02' '09:34:54'
 '09:00:32' '09:36:55' '10:10:38' '13:01:52' '09:33:38' '14:46:01'
 '13:10:03' '11:37:32' '11:07:09' '13:39:02' '08:09:49' '18:47:31'
 '21:24:52' '14:07:46' '09:00:31' '12:00:30' '11:06:15' '13:01:16'
 '17:33:18' '21:49:51' '19:04:54' '09:31:56']
        hora franja_horaria
0   09:26:29           9:30
1   19:47:30          19-20
2   12:51:46          13:00
3   16:13:56          14:00
4   09:59:46          10:00
5   17:00:11          17-18
6   11:12:48          11:00
7   09:26:20           9:30
8   17:01:16          17-18
9   09:24:03           9:30
10  08:45:49           9:00
11  09:16:59           9:30
12  09:13:08           9:00
13  10:36:33          11:00
14  08:36:29  antes de 8:45
15  10:59:52          11:00
16  09:51:02          10:00
17  09:34:54           9:30
18  09:00:32        

In [32]:
ct_data_copy = ct_data_copy.drop('caption', axis=1)
ct_data_copy = ct_data_copy.rename(columns={"type": "tipo"})
ct_data_copy = ct_data_copy.drop('hashtags_list', axis=1)
ct_data_copy.to_csv(r"C:\Users\Yeray\Desktop\DATA_SCIENCE_ML\IG-Engagemente-Insights\IG-Engagement-Insights\ANALYSIS_IG_ACCOUNT\DATA_CT\CT_SOLO_FINAL_LISTO.csv", index=False, encoding='utf-8-sig')


In [33]:
print(tabulate(ct_data_copy.head(10), headers=ct_data_copy.columns, tablefmt='fancy_grid'))

╒════╤═════════════════╤══════════════╤═════════╤════════════╤════════════════╤══════════════╤══════════════╤═══════════╤════════╤════════════╤══════════╤══════╤══════════════════╕
│    │   commentsCount │   likesCount │ tipo    │   hasMusic │   num_hashtags │   num_images │ dia_semana   │ mes       │   year │ fecha      │ hora     │   id │ franja_horaria   │
╞════╪═════════════════╪══════════════╪═════════╪════════════╪════════════════╪══════════════╪══════════════╪═══════════╪════════╪════════════╪══════════╪══════╪══════════════════╡
│  0 │              16 │           66 │ Sidecar │          0 │              0 │            5 │ Jueves       │ Febrero   │   2025 │ 06-02-2025 │ 09:26:29 │    0 │ 9:30             │
├────┼─────────────────┼──────────────┼─────────┼────────────┼────────────────┼──────────────┼──────────────┼───────────┼────────┼────────────┼──────────┼──────┼──────────────────┤
│  1 │              11 │           84 │ Sidecar │          0 │              9 │            2 │ 